In [75]:
import pandas as pd
import requests
import folium as fl 
from folium.plugins import MarkerCluster
import matplotlib.pyplot as plt 
import seaborn as sns
import dash
from io import StringIO

In [76]:
url='https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/datasets/spacex_launch_geo.csv'
response=requests.get(url)
data=response.text
df=pd.read_csv(StringIO(data))

In [77]:
df.tail()

,Flight Number,Date,Time (UTC),Booster Version,Launch Site,Payload,Payload Mass (kg),Orbit,Customer,Landing Outcome,class,Lat,Long
51,48,2018-01-31,21:25:00,F9 FT B1032.2,CCAFS SLC-40,GovSat-1 / SES-16,4230.0,GTO,SES,Controlled (ocean),0,28.563197,-80.57682
52,50,2018-03-06,5:33:00,F9 B4 B1044,CCAFS SLC-40,Hispasat 30W-6 PODSat,6092.0,GTO,Hispasat NovaWurks,No attempt,0,28.563197,-80.57682
53,52,2018-04-02,20:30:00,F9 B4 B1039.2,CCAFS SLC-40,SpaceX CRS-14,2647.0,LEO (ISS),NASA (CRS),No attempt,0,28.563197,-80.57682
54,53,2018-04-18,22:51:00,F9 B4 B1045.1,CCAFS SLC-40,Transiting Exoplanet Survey Satellite (TESS),362.0,HEO,NASA (LSP),Success (drone ship),1,28.563197,-80.57682
55,56,2018-06-04,4:45:00,F9 B4 B1040.2,CCAFS SLC-40,SES-12,5384.0,GTO,SES,No attempt,0,28.563197,-80.57682


In [78]:
df_launches=df[['Launch Site','Lat','Long','class']]

In [79]:
df_launches.head()

,Launch Site,Lat,Long,class
0,CCAFS LC-40,28.562302,-80.577356,0
1,CCAFS LC-40,28.562302,-80.577356,0
2,CCAFS LC-40,28.562302,-80.577356,0
3,CCAFS LC-40,28.562302,-80.577356,0
4,CCAFS LC-40,28.562302,-80.577356,0


In [80]:
mapa=fl.Map(location=[df_launches['Lat'].mean(),df_launches['Long'].mean()],zoom_start=4)

In [81]:
for i, row in df.iterrows():

    
    fl.Marker(
        location=(row['Lat'], row['Long']),
        popup=row['Launch Site'],
        icon=fl.Icon(color='blue')
    ).add_to(mapa)
    
    fl.Circle(
        location=(row['Lat'], row['Long']),
        color='blue',
        fill=True,
        fill_color='blue',
        radius=11,
        weight=1
    ).add_to(mapa)


In [82]:
marker_cluster = MarkerCluster().add_to(mapa)

# Iterar sobre el DataFrame y agregar marcadores al cluster
for i, row in df.iterrows():
    if row['class'] == 1:
        color = 'green'
        popup_text = f"Launch Site: {row['Launch Site']} - Success"
    else:
        color = 'red'
        popup_text = f"Launch Site: {row['Launch Site']} - Failure"
    
    fl.Marker(
        location=(row['Lat'], row['Long']),
        popup=popup_text,
        icon=fl.Icon(color=color)
    ).add_to(marker_cluster)

In [83]:
mapa

In [84]:
from folium.plugins import MousePosition

In [85]:
MousePosition().add_to(mapa)

In [86]:
from math import sin, cos, sqrt, atan2, radians

In [87]:
def calculate_distance(lat1, lon1, lat2, lon2):
    # approximate radius of earth in km
    R = 6373.0

    lat1 = radians(lat1)
    lon1 = radians(lon1)
    lat2 = radians(lat2)
    lon2 = radians(lon2)

    dlon = lon2 - lon1
    dlat = lat2 - lat1

    a = sin(dlat / 2)**2 + cos(lat1) * cos(lat2) * sin(dlon / 2)**2
    c = 2 * atan2(sqrt(a), sqrt(1 - a))

    distance = R * c
    return distance


In [88]:
mapa